##Importações necessárias

In [1]:
import numpy as np
import pandas as pd

## Leitura e tratamento dos dados

In [2]:
#Leitura da base de dados como um DataFrame, adicionando um header, já que não há inicialmente
data = pd.read_csv('banana.csv', sep='\t', names=['Values'])
#Criando um DataFrame só com as informações que são dadas inicialmente (o que se inicia com "@")
headers = data[data["Values"].str.contains(r"@")]
#Eliminando todo o cabeçalho e mantendo somente os pontos
only_values = data[data["Values"].str.contains("@")==False]
#Transformando o DataFrame em List
list_only_values = only_values.values.tolist()
#Permutando os pontos
random_sort = np.random.permutation(list_only_values)
#Como os pontos foram lidos como strings, separo cada elemento entre as vírgulas
random_sort = [x[0].split(",") for x in random_sort]
#Transformo a coordenada dos pontos em floats e mantenho a classe como string
for row in range(len(random_sort)):
  for i in range(len(random_sort[row])):
    if random_sort[row][i] != random_sort[row][-1]:
      random_sort[row][i] = float(random_sort[row][i])
#Junto os floats em uma lista, pois são a coordenada do ponto. A classe fica em outra lista
random_sort = [[x[:-1],x[-1]] for x in random_sort]
#Divido os dados entre treino e teste respeitando a proporção 70/30
training = random_sort[:int(len(random_sort)*0.7)]
testing = random_sort[int(len(random_sort)*0.7):]
print(testing)
#A dimensão do espaço é definida pelo tamanho da lista de coordenadas
dimension = len(testing[0])

[[[-0.664, 1.18], '-1.0'], [[-0.386, 1.5], '-1.0'], [[-1.2, 0.455], '-1.0'], [[0.409, 1.76], '-1.0'], [[-0.048, -1.06], '-1.0'], [[0.955, -0.0158], '-1.0'], [[0.137, 1.69], '-1.0'], [[0.073, -0.00169], '1.0'], [[-0.959, 1.08], '-1.0'], [[-1.12, 1.1], '-1.0'], [[-0.946, 1.02], '-1.0'], [[1.12, 0.173], '-1.0'], [[-1.12, -1.2], '-1.0'], [[0.154, 0.629], '1.0'], [[-0.192, 1.66], '-1.0'], [[-1.57, 0.629], '-1.0'], [[0.317, -1.13], '-1.0'], [[0.122, 0.599], '1.0'], [[-1.15, -0.659], '1.0'], [[0.0438, 0.536], '1.0'], [[-0.076, -1.12], '-1.0'], [[0.595, 0.546], '1.0'], [[1.81, -0.0547], '-1.0'], [[-0.737, -0.0677], '1.0'], [[-1.32, -0.403], '1.0'], [[0.281, -0.4], '1.0'], [[-0.621, 1.16], '-1.0'], [[-1.77, 1.01], '-1.0'], [[-0.338, -0.91], '-1.0'], [[0.524, -0.648], '1.0'], [[0.487, 1.45], '-1.0'], [[0.123, -0.283], '1.0'], [[0.105, -0.208], '1.0'], [[0.866, 0.612], '-1.0'], [[-1.34, 0.805], '-1.0'], [[-0.8, -0.282], '1.0'], [[-0.184, -0.751], '1.0'], [[-1.8, -1.62], '1.0'], [[1.26, -0.912], '

##Classe Point: é criado um ***tipo ponto*** para auxiliar na criação da KDTree

In [3]:
class Point:
  def __init__(self, value, left, right, depth, dim):
    #Valor dos pontos
    self.value = value
    #Para qual nó da esquerda o nó atual aponta
    self.left = left
    #Para qual nó da direita o nó atual aponta
    self.right = right
    #A profundidade que o nó se encontra na árvore KD
    self.depth = depth
    #Dimensão do espaço
    self.dim = dim

##Árvore KD:

In [4]:
class KDTree:
  #Construtor
  def __init__(self):
    self.KDTree = 'KDTree'

  def EvenMedian(self, sorted_data):
    e_median = (sorted_data[len(sorted_data)//2] + sorted_data[(len(sorted_data)//2) + 1])//2
    return e_median

  def OddMedian(self, sorted_data):
    o_median = sorted_data[len(sorted_data)//2]
    return o_median
                            
  def BuildKDTree(self, data, depth, dim):
    #Se resta apenas um ponto na construção, é uma folha
    if len(data) == 1:
      leaf = Point(data, None, None, depth, dim)
      return leaf
    else:
      #O resto da divisão da profundidade pela dimensão do ponto garante profundidades cíclicas
      #Ordenação dos dados baseado nesse critério cíclico
      sorted_data = sorted(data, key=lambda x: x[0][depth % dim])
      #Calculando a mediana
      if len(sorted_data) % 2 == 0:
        median = EvenMedian(sorted_data)
      else:
        median = OddMedian(sorted_data)
      #Criando as subdivisões  
      sub_left = sorted_data[:median,:]
      sub_right = sorted_data[median:,:]

    v_left = self.BuildKDTree(sub_left, depth + 1, dim)
    v_right = self.BuildKDTree(sub_right, depth + 1, dim)
    
    root = Point(median, v_left, v_right, depth, dim)
    return root